## Spam mail分類を行う

In [15]:

%matplotlib inline
""" spam を分類する　"""
import libs.load_spam_data
import importlib
importlib.reload(libs.load_spam_data)
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, losses, optimizers
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, Input, Embedding, GlobalAveragePooling1D

# データの読み込み
FILE_PATH = './spam_data/spam.csv'
train_tokes, train_labels, test_tokes, test_labels = libs.load_spam_data.load_spam_data(FILE_PATH)
print(f"Train tokens: {len(train_tokes)}")
print(f"Test tokens: {len(test_tokes)}")
print("train #1000:", train_tokes[1000])
print("train label #1000:", train_labels[1000])

max_train_len = max([len(x) for x in train_tokes])
#print("max_train_len = ", max_train_len)
max_test_len = max([len(x) for x in test_tokes])
#print("max_test_len = ", max_test_len)

x_train  = np.array(pad_sequences(train_tokes, maxlen=max_train_len, padding='post'))
x_test   = np.array(pad_sequences(test_tokes, maxlen=max_test_len, padding='post'))
y_labels = np.array(train_labels)
y_test   = np.array(test_labels)
#print("x_train[1000]:", x_train[1000])
#print("x_test[1000]:", x_test[1000])

max_train_vocab = 0
for token in train_tokes:
    for y in token :
        if y > max_train_vocab:
            max_train_vocab = y
print("max_train_vocab = ", max_train_vocab)
max_test_vocab = 0
for token in test_tokes:
    for y in token :
        if y > max_test_vocab:
            max_test_vocab = y
print("max_test_vocab = ", max_test_vocab)
if max_train_vocab > max_test_vocab:
    max_voc = max_train_vocab
else:
    max_voc = max_test_vocab
print("max_voc = ", max_voc)

# modelの定義
enbedding_dim = 128
model       = Sequential()
m_input     =  Input(shape=(None,), dtype='int32')
m_enbedding = Embedding(input_dim=max_voc+1, output_dim=enbedding_dim, input_length=max_train_len)
m_global    = GlobalAveragePooling1D()
m_dense_1   = Dense(enbedding_dim, activation='relu')
m_dense_2   = Dense(1, activation='sigmoid')

model.add(m_input)
model.add(m_enbedding)
model.add(m_global)
model.add(m_dense_1)
model.add(m_dense_2)

model.summary

model.compile(optimizer='adam', loss=losses.BinaryCrossentropy(), metrics=['accuracy'])

model.summary()

# 学習  
BATCH_SIZE = 128
NUM_EPOCHS = 10
history = model.fit(x_train, y_labels,
                    epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_data=(x_test, y_test),
                    verbose=1)

# x_test中のspamデータがどれだけ見つけられたか　：　評価基準
res = model.predict(x_test)
threshhold = 0.5
spam_in_test = sum(y_test)
print("spam # in y_test = ", spam_in_test)
#print("res[0:10]:", res[0:10])
num_ok, num_ng = 0, 0
for i in range(res.shape[0]):
    if res[i] > threshhold:
        if y_test[i] == 1:
            num_ok += 1
        else:
            num_ng += 1
print("perfomance to found out spam mail = ", num_ok / spam_in_test)
print("# fals negative = ", num_ng)

[nltk_data] Downloading package punkt to /Users/nisida/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nisida/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2.0.2
-------------
hum count   =  4825
supam count =  747
-------------
max_token_len =  171
[665, 202, 221, 666, 667, 668, 595, 477, 21, 669, 294, 670, 553]
Train tokens: 4457
Test tokens: 1114
train #1000: [3517, 3518, 647, 2176]
train label #1000: 0
max_train_vocab =  8906
max_test_vocab =  10124
max_voc =  10124
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, None, 128)         1296000   
                                                                 
 global_average_pooling1d_1  (None, 128)               0         
 4 (GlobalAveragePooling1D)                                      
                                                                 
 dense_28 (Dense)            (None, 128)               16512     
                                                                 
 dense_29 (Dense)            (None, 1)                 129      

,{'name': 1, 'price':2} ->  all_token内の全ての単語にユニークなID（数字)をふる（Pythonの辞書で作成する）. -> コーパスと呼ぶ。

気になるpoint

token1 = [1,3, 6, 7, 0, 0] , token2 = [10, 13, 14, 15 16, 17]


In [ ]:
import nltk 

print(nltk.__version__)